# Cellular Imaging Demo

We will be using the Malaria dataset in this demo.

The Malaria dataset contains a total of 27,558 cell images with equal instances of parasitized and uninfected cells from the thin blood smear slide images of segmented cells.

## Step 1

## Repository Cloning and Setup

In [ ]:
#copy the tensor flow repo from git hub 

!git clone https://github.com/tensorflow/models.git \
  --branch=v2.1.0 \
  --depth=1

In [ ]:
#move the trainer folder inside the model folder

!cp -r trainer  models/

In [ ]:
# Change the current working directory to models
import os
os.chdir('models/')

os.getcwd()

In [ ]:
# Create the setup.py file

In [ ]:
%%writefile setup.py
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES=['tensorflow-datasets~=3.1', 
                   'pip>=20.2',
                   'absl-py<0.9,>=0.7']

setup(
    name='official',
    _version_='0.1',
    install_requires=REQUIRED_PACKAGES,
    include_package_data=True,
    packages=find_packages()
)

# Step 2

## Training the model on the AI platform

In [ ]:
%env PROJECT_ID=mwpmltr
%env BUCKET_ID=bipin_workshop_bucket
%env REGION=us-central1
%env DATA_DIR= temp_data_bukcet

In [ ]:
now = !date +"%Y%m%d_%H%M%S"
%env JOB_NAME=cellular_img__CPU$now.s

!gcloud ai-platform jobs submit training $JOB_NAME \
  --package-path trainer \
  --module-name trainer.trainer  \
  --region $REGION \
  --python-version 3.7 \
  --runtime-version 2.1 \
  --staging-bucket gs://${BUCKET_ID} \
  -- \
  --tpu local \
  --model_dir gs://${BUCKET_ID}/cellular_img__CPU_model_files \
  --data_dir gs://${DATA_DIR} \
  --train_epochs 1 \
  --distribution_strategy off \
  --num_gpus 0 \
  --download False

                
# Stream logs so that training is done before subsequent cells are run.
# Remove  '> /dev/null' to see step-by-step output of the model build steps.
!gcloud ai-platform jobs stream-logs $JOB_NAME > /dev/null

# Model should exit with status "SUCCEEDED"
!gcloud ai-platform jobs describe $JOB_NAME --format="value(state)"

# Step 3

### Train the model on the AI Platform Using the TPU

#### Prerequisites before using a Cloud TPU
#### Authorizing your Cloud TPU to access your project 

[TPU Environment Setup](https://cloud.google.com/ai-platform/training/docs/using-tpus#console) - These steps need to be executed by the Data Enginner for setting up the environment to use the TPU's. 

In [ ]:
now = !date +"%Y%m%d_%H%M%S"
%env JOB_NAME=Cellular_img_TPU$now.s

!gcloud ai-platform jobs submit training $JOB_NAME \
  --scale-tier BASIC_TPU \
  --package-path trainer \
  --module-name trainer.trainer  \
  --region $REGION \
  --python-version 3.7 \
  --runtime-version 2.1 \
  --staging-bucket gs://${BUCKET_ID} \
  -- \
  --model_dir gs://${BUCKET_ID}/cellular_img__TPU_model_files \
  --data_dir gs://${DATA_DIR} \
  --train_epochs 1 \
  --distribution_strategy tpu \
  --download False

                
# Stream logs so that training is done before subsequent cells are run.
# Remove  '> /dev/null' to see step-by-step output of the model build steps.
!gcloud ai-platform jobs stream-logs $JOB_NAME > /dev/null

# Model should exit with status "SUCCEEDED"
!gcloud ai-platform jobs describe $JOB_NAME --format="value(state)"